In [1]:
import torch, torchtune, yaml
from torchtune import config, utils
from generate import InferenceRecipe
from omegaconf import DictConfig
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
from test_data import test_set

In [3]:
# adapter = torch.load('checkpoints/Llama-3-LLaMS-kowiki-maxlenseq1024/adapter_9.pt')

In [4]:
# weights = torch.load('checkpoints/Llama-3-LLaMS-kowiki-maxlenseq1024/meta_model_9.pt')

In [5]:
config_base = './configs/llams_base_generation_kowiki_20240520__.yaml'

In [6]:
config_lists = [
    {
        'checkpoint_dir': 'checkpoints/Meta-Llama-3-8B/original',
        'checkpoint_files': ['consolidated.00.pth'],
        'output_dir': 'checkpoints/Meta-Llama-3-8B/original'
    },
    {
        'checkpoint_dir': 'checkpoints/Llama-3-LLaMS-kowiki',
        'checkpoint_files': ['meta_model_0.pt'],
        'output_dir': 'checkpoints/Llama-3-LLaMS-kowiki'
    },
    {
        'checkpoint_dir': 'checkpoints/Llama-3-LLaMS-kowiki',
        'checkpoint_files': ['meta_model_1.pt'],
        'output_dir': 'checkpoints/Llama-3-LLaMS-kowiki'
    },
    {
        'checkpoint_dir': 'checkpoints/Llama-3-LLaMS-kowiki',
        'checkpoint_files': ['meta_model_2.pt'],
        'output_dir': 'checkpoints/Llama-3-LLaMS-kowiki'
    },
    {
        'checkpoint_dir': 'checkpoints/Llama-3-LLaMS-kowiki',
        'checkpoint_files': ['meta_model_3.pt'],
        'output_dir': 'checkpoints/Llama-3-LLaMS-kowiki'
    },
    {
        'checkpoint_dir': 'checkpoints/Llama-3-LLaMS-kowiki',
        'checkpoint_files': ['meta_model_4.pt'],
        'output_dir': 'checkpoints/Llama-3-LLaMS-kowiki'
    },
]

In [7]:
results = []
for config_item in config_lists:
    with open(config_base) as f:
        config_dict = DictConfig(yaml.safe_load(f))
    for k, v in config_item.items():
        config_dict.checkpointer[k] = v
        
    a = InferenceRecipe(config_dict)
    a.setup(config_dict)

    result = []
    for input_, _ in test_set:
        config_dict.prompt = input_
        res = a.generate(config_dict)
        result.append(res)

    results.append(result)
    del a
    torch.cuda.empty_cache()

DEBUG:torchtune.utils.logging:Setting manual seed to local seed 1234. Local seed is seed + rank = 1234 + 0
INFO:torchtune.utils.logging:Model is initialized with precision torch.bfloat16.
INFO:torchtune.utils.logging:어려운 상황에서도 끝까지 포기하지 않는 사람은 누구인가? 그들은 어떤 사람인가? 그들은 어떤 생각을 가지고 있는가? 그들은 어떤 행동을 하는가? 그들은 어떤 결과를 얻는가? 그들은 어떤 사람인가? 그들은 어떤 생각을 가지고 있는가? 그들은 어떤 행동을 하는가? 그들은 어떤 결과를 얻는가? 그들은 어떤 사람인가? 그들은 어떤 생각을 가지고 있는가? 그들은 어떤 행동을 하는가? 그들은 어떤 결과를 얻는가? 그들은 어떤 사람인가? 그들은 어떤 생각을 가지고 있는가? 그들은 어떤 행동을 하는가? 그들은 어떤 결과를 얻는가? 그들은 어떤 사람인가? 그들은 어떤 생각을 가지고 있는가? 그들은 어떤 행동을 하는가? 그들은 어떤 결과를 얻는가? 그들은 어떤 사람인가? 그들은 어떤 생각을 가지고 있는가? 그들은 어떤 행동을 하는가? 그들은 어떤 결과를 얻는가? 그들은 어떤 사람인가? 그들은 어떤 생각을 가지고 있는가? 그들은 어떤 행동을 하는가? 그들은 어떤 결과를 얻는가? 그들은 어떤 사람인가? 그들은 어떤 생각을 가지고 있는가? 그들은 어떤 행동을 하는가? 그들은 어떤 결과를 얻는가? 그들은 어떤 사람인가? 그들은 어떤 생각을 가지고 있는가? 그들은 어떤 행동을 하는가? 그들은 어떤 결과를 얻는가? 그들은 어떤 사람인가? 그들은 어떤
INFO:torchtune.utils.logging:Time for inference: 16.33 sec total, 18.37 tokens/sec
INFO:torchtune.utils.logging:Bandwidth achieved: 335.73 GB/s
IN

```
results = []
for config_item in config_lists:
    with open(config_item) as f:
        config_dict = DictConfig(yaml.safe_load(f))
    a = InferenceRecipe(config_dict)
    a.setup(config_dict)

    result = []
    for input_, _ in test_set:
        config_dict.prompt = input_
        res = a.generate(config_dict)
        result.append(res)

    results.append(result)
    del a
    torch.cuda.empty_cache()
```

In [8]:
input_ = [x[0] for x in test_set]

In [9]:
epoch_results = {}
for idx, result in enumerate(results):
    epoch_result = []
    for x, y in zip(input_, result):
        epoch_result.append(y[len(x):])
    epoch_results[f'Epoch {idx:02d}'] = epoch_result

In [10]:
df = pd.DataFrame()

In [11]:
df['Input'] = input_
for k, v in epoch_results.items():
    df[k] = v

In [12]:
df.to_csv('test_20240909.csv')

```
from torchtuning import kowiki_dataset
from torchtune.models.llama3 import llama3_tokenizer
tokenizer = llama3_tokenizer('checkpoints/Meta-Llama-3-8B/original/tokenizer.model')
d = kowiki_dataset(
    tokenizer=tokenizer,
    max_seq_len=1536,
    data_files='data/20240520/kowikitext_20240520.csv',
    split='train',
)
```

In [13]:
df

,Input,Epoch 00,Epoch 01,Epoch 02,Epoch 03,Epoch 04,Epoch 05
0,어려운 상황에서도 끝까지 포기하지 않는 사람은,누구인가? 그들은 어떤 사람인가? 그들은 어떤 생각을 가지고 있는가? 그들은 어떤...,", 결국 성공할 것이다.\n어려운 상황에서도 끝까지 포기하지 않는 사람은, 결국 성...",누구인가?,누구인가?,누구인가?,누구인가?
1,아무리 작은 노력이라도 꾸준히 하면,큰 변화가 일어날 수 있다.\nI have been trying to get ba...,큰 변화가 일어난다.\nI am a person who is always tryi...,큰 변화가 일어난다.\nI am a person who is always tryi...,큰 변화가 생긴다.\nI am a person who is always tryin...,큰 변화가 생긴다.\nI am a big fan of your blog and t...,큰 변화가 생긴다.\nI am a big fan of your blog and t...
2,먼 길을 가기 위해서는,먼저 출발을 해야 한다. 출발을 하기 위해서는 준비가 필요하다. 준비를 하기 위해...,\n먼 길을 가기 위해서는\nI have to go a long way to go\...,먼저 출발을 해야 한다.,\n먼 길을 가기 위해서는\nI have to go a long way to go\...,\n먼 길을 가기 위해서는\nI have a dream that my four li...,\n먼 길을 가기 위해서는\nI have a dream that my four li...
3,끝없이 펼쳐진 바다 위에,"있는 섬, 그 섬 위에 있는 마을, 그 마을에 사는 사람들. 그 사람들 중 한 명...","떠 있는 섬, 그 섬 위에 있는 마을, 그 마을에 사는 사람들, 그 사람들에 대한...","떠 있는 섬, 그 섬 위에 떠 있는 집, 그 집 위에 떠 있는 사람들.","떠 있는 섬, 그 섬 위에 떠 있는 집, 그 집 위에 떠 있는 사람, 그 사람의 ...","떠다니는 섬은, 그 섬의 주민들이 섬을 떠나지 못하게 하는 장벽이기도 하다.","떠다니는 섬은, 그 섬의 주민들이 섬을 떠나지 못하게 하는 장벽이기도 하다."
4,푸른 하늘 아래,서\n푸른 하늘 아래서\nBlue Sky Under\nBlue Sky Under\n...,서\n푸른 하늘 아래서 푸른 하늘 아래서 푸른 하늘 아래서 푸른 하늘 아래서 푸른 ...,서\n푸른 하늘 아래서 푸른 하늘 아래서 푸른 하늘 아래서 푸른 하늘 아래서 푸른 ...,서\nThe Blue Sky\nThe Blue Sky\nThe Blue Sky is...,", 푸른 바다 위\n푸른 하늘 아래, 푸른 바다 위\n푸른 하늘 아래, 푸른 바다 ...",", 푸른 바다 위\n푸른 하늘 아래, 푸른 바다 위\n푸른 하늘 아래, 푸른 바다 ..."
5,고요한 저녁노을이 지고,\n고요한 저녁노을이 지고\n고요한 저녁노을이 지고\n고요한 저녁노을이 지고\n고요...,\n고요한 저녁노을이 지고\n고요한 저녁노을이 지고\n고요한 저녁노을이 지고\n고요...,\n고요한 저녁노을이 지고\n고요한 저녁노을이 지고\n고요한 저녁노을이 지고\n고요...,\n고요한 저녁노을이 지고\n고요한 저녁노을이 지고\n고요한 저녁노을이 지고\n고요...,\n그리고 그저 그저 그저 그저\n그저 그저 그저 그저 그저 그저\n그저 그저 그저...,\n그리고 그저 그저 그저 그저\n그저 그저 그저 그저 그저 그저\n그저 그저 그저...
6,깊은 숲속 작은 오두막에는 마법사 할아버지가 살고 있었다.,"그는 오래전부터 마법을 연구해왔고, 그 결과로 많은 마법의 힘을 얻었다. 하지만 ...","그는 마법을 잘 써서 많은 사람들에게 도움을 주었다. 어느 날, 마법사 할아버지가...","그는 마법을 잘 써서, 그의 집은 항상 따뜻하고, 그의 밥상은 항상 푸짐했다. 그...","그는 마법을 잘 써서, 그의 집은 항상 따뜻하고, 그의 밥상은 항상 푸짐하고, 그...","그는 마법을 잘 알고 있었고, 마법을 잘 써서, 그의 집은 항상 따뜻하고, 그의 ...","그는 마법을 잘 알고 있었고, 마법을 잘 써서, 그의 집은 항상 따뜻하고, 그의 ..."
7,어느 날 용감한 왕자는 멀리 떨어진 성을 구하기 위해 긴 여행을 떠났다.,왕자는 길을 가다가 숲 속에서 한 마리의 용을 만났다. 용은 왕자를 공격하기 위해...,"왕자는 길을 가다가 길을 잃고, 길을 잃은 왕자는 길을 찾기 위해 길을 따라가기 ...","왕자는 길을 가다가 길을 잃고, 길을 잃은 왕자는 길을 찾기 위해 길을 따라가기 ...","왕자는 길을 가다가 길을 잃고, 길을 잃은 왕자는 길을 찾기 위해 길을 따라가기 ...","왕자는 길을 가면서 많은 위험을 겪었고, 결국 왕자는 성에 도착했다. 왕자는 성에...","왕자는 길을 가면서 많은 위험을 겪었고, 결국 왕자는 성에 도착했다. 왕자는 성에..."
8,옛날 옛적 한 마을에는 착한 마음씨를 가진 소녀가 살고 있었다.,"어느 날, 소녀는 마을에 큰 불이 나서 마을이 모두 불타버린 것을 보고, 소녀는 ...","그녀는 마을 사람들에게 사랑받는 소녀였고, 마을 사람들은 그녀를 사랑하고 존경했다...","어느 날, 소녀는 마을에 찾아온 외로운 늙은 할아버지와 함께 산을 오르며 산책을 ...","어느 날, 소녀는 마을에 찾아온 외로운 늙은 할아버지와 친해졌다. 할아버지가 소녀...",그 소녀는 마을 사람들에게 사랑받고 있었다. 어느 날 소녀는 마을 사람들과 함께 ...,그 소녀는 마을 사람들에게 사랑받고 있었다. 어느 날 소녀는 마을 사람들과 함께 ...
9,인생은 한 번뿐이다.,(Life is only once)\n인생은 한 번뿐이다. (Life is onl...,그러니 인생을 즐기자.,그 한 번의 인생을 잘 살기 위해서는 무엇보다도 자신을 잘 알아야 한다. 자신을 ...,그러니 인생을 즐기자.,"그 한 번의 인생을 살면서, 어떤 일을 하고 싶은가? 어떤 일을 하고 싶지 않아?...","그 한 번의 인생을 살면서, 어떤 일을 하고 싶은가? 어떤 일을 하고 싶지 않아?..."
